In [1]:
!pip install tensorflow numpy matplotlib scikit-learn pillow pandas scikeras

INFO: pip is looking at multiple versions of scikeras to determine which version is compatible with other requirements. This could take a while.


imports various libraries and modules. It includes TensorFlow and Keras for deep learning model creation, preprocessing tools, layer definitions, callbacks, and optimization. Scikeras is used for integrating Keras models with Scikit-learn. Additionally, it imports logging for tracking the progress.

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.optimizers import Adam ,RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import logging


2024-06-22 01:14:14.403781: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 01:14:47.173861: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 01:14:47.173928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 01:14:51.569321: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-22 01:14:59.321058: I tensorflow/core/platform/cpu_feature_guar

random seed for NumPy and TensorFlow to ensure reproducibility

In [3]:
np.random.seed(42)
tf.random.set_seed(42)

ets up directories for training, testing, and validation data. It creates image data generators for training and testing datasets with specified augmentations for the training data to improve generalization. The flow_from_directory method is used to load images from directories into the generators.

In [9]:
train_dir = 'images/training'
test_dir = 'images/testing'
val_dir = 'images/testing'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


val_datagen = ImageDataGenerator(rescale=1.0/255.0)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 2393 images belonging to 3 classes.
Found 597 images belonging to 3 classes.
Found 597 images belonging to 3 classes.


create_baseline_model to build a baseline CNN model. It specifies input shape, number of classes, and learning rate. It includes convolutional, pooling, flatten, and dense layers with optional batch normalization and dropout for regularization. The model is compiled with an optimizer, loss function, and metrics.

In [5]:
def create_baseline_model(input_shape=(224, 224, 3), num_classes=3, learning_rate=0.001, l2_reg=0.001, activation='relu', optimizer='adam'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=input_shape, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation=activation),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    optimizer_instance = tf.keras.optimizers.get(optimizer)
    optimizer_instance.learning_rate = learning_rate

    model.compile(optimizer=optimizer_instance, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Intializing call backs

In [6]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
Initializes base line  models

In [7]:
baseline_model = create_baseline_model()

2024-06-22 01:19:22.933738: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-22 01:19:39.185716: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-22 01:19:39.185995: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
history_baseline = baseline_model.fit(train_generator, validation_data=test_generator, epochs=5, callbacks=[reduce_lr, early_stopping])

Epoch 1/5


2024-06-22 01:21:00.620103: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-06-22 01:21:39.312510: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fee815fad10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-22 01:21:39.312567: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-06-22 01:21:39.519356: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1719019299.670677   13870 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


75/75 [==============================] - 398s 5s/step - loss: 3.3975 - accuracy: 0.6389 - val_loss: 2.8756 - val_accuracy: 0.5812 - lr: 0.0010
Epoch 2/5
75/75 [==============================] - 55s 734ms/step - loss: 2.5764 - accuracy: 0.7681 - val_loss: 2.7037 - val_accuracy: 0.6734 - lr: 0.0010
Epoch 3/5
75/75 [==============================] - 54s 724ms/step - loss: 2.0823 - accuracy: 0.7948 - val_loss: 4.0674 - val_accuracy: 0.4858 - lr: 0.0010
Epoch 4/5
75/75 [==============================] - 55s 729ms/step - loss: 1.7310 - accuracy: 0.8061 - val_loss: 1.9993 - val_accuracy: 0.6717 - lr: 0.0010
Epoch 5/5
75/75 [==============================] - 54s 717ms/step - loss: 1.4854 - accuracy: 0.8253 - val_loss: 3.2697 - val_accuracy: 0.4858 - lr: 0.0010




Importing CosineDecay:
Imports the `CosineDecay` class from TensorFlow Keras to implement a learning rate schedule that decays following a cosine curve, optimizing training efficiency.

Setting Parameters:
Defines the batch size, number of epochs, input image dimensions, and number of output classes, which are crucial for configuring the training process.

Data Augmentation:
Configures data augmentation for the training images to enhance model generalization and normalizes pixel values for both training and validation images to ensure consistent input data.

Creating Simplified ResNet50 Model:
Defines a simplified ResNet50 model by adding custom layers on top of a pre-trained base, freezing the base layers, and setting up a cosine decay schedule for the learning rate to optimize training.

Training the Model:
Creates and compiles the ResNet50 model, sets up early stopping to prevent overfitting, and trains the model using the training and validation data, with progress monitored by validation loss.

In [11]:

from tensorflow.keras.optimizers.schedules import CosineDecay

batch_size = 32
epochs = 5
input_shape = (224, 224, 3)
num_classes = 3

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Simplified ResNet50 model
def create_simplified_resnet50_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    # Learning rate schedule
    initial_learning_rate = 0.001
    lr_schedule = CosineDecay(
        initial_learning_rate,
        decay_steps=1000,
        alpha=0.1
    )
    
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create and train the ResNet50 model
resnet50_model = create_simplified_resnet50_model()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

resnet50_history = resnet50_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    callbacks=[early_stopping]
)


94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/5
74/74 [==============================] - 63s 744ms/step - loss: 1.3440 - accuracy: 0.4795 - val_loss: 0.7769 - val_accuracy: 0.5069
Epoch 2/5
74/74 [==============================] - 54s 723ms/step - loss: 0.7651 - accuracy: 0.5125 - val_loss: 0.7434 - val_accuracy: 0.4861
Epoch 3/5
74/74 [==============================] - 54s 725ms/step - loss: 0.7337 - accuracy: 0.5091 - val_loss: 0.7274 - val_accuracy: 0.4878
Epoch 4/5
74/74 [==============================] - 53s 712ms/step - loss: 0.7247 - accuracy: 0.4998 - val_loss: 0.7138 - val_accuracy: 0.5122
Epoch 5/5
74/74 [==============================] - 54s 724ms/step - loss: 0.7163 - accuracy: 0.5066 - val_loss: 0.7088 - val_accuracy: 0.5191


Data Augmentation:
Configures data augmentation for the training images to enhance model generalization by applying various transformations like rescaling, shearing, zooming, horizontal flipping, rotation, and shifting. The validation images are only rescaled.

Simplified EfficientNetB0 Model Creation:
Defines a simplified EfficientNetB0 model by adding custom layers on top of the pre-trained EfficientNetB0 base, including global average pooling, dense, and dropout layers. The base model's layers are frozen to retain pre-trained weights.

Learning Rate Schedule:
Implements a cosine decay schedule for the learning rate, starting at an initial value and gradually reducing it following a cosine curve to optimize the training process.

Training the EfficientNetB0 Model:
Creates and compiles the EfficientNetB0 model, sets up early stopping to monitor validation loss and prevent overfitting, and trains the model using the training and validation data generators, with progress monitored by validation performance.

In [13]:


# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Simplified EfficientNetB0 model
def create_simplified_efficientnet_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    # Learning rate schedule
    initial_learning_rate = 0.001
    lr_schedule = CosineDecay(
        initial_learning_rate,
        decay_steps=1000,
        alpha=0.1
    )
    
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create and train the EfficientNetB0 model
efficientnet_model = create_simplified_efficientnet_model()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

efficientnet_history = efficientnet_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    callbacks=[early_stopping]
)


Epoch 1/5


2024-06-22 01:38:19.018748: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


74/74 [==============================] - 61s 742ms/step - loss: 1.7788 - accuracy: 0.4972 - val_loss: 1.0501 - val_accuracy: 0.5208
Epoch 2/5
74/74 [==============================] - 56s 750ms/step - loss: 0.9071 - accuracy: 0.5066 - val_loss: 0.8024 - val_accuracy: 0.5139
Epoch 3/5
74/74 [==============================] - 53s 719ms/step - loss: 0.7757 - accuracy: 0.4994 - val_loss: 0.7406 - val_accuracy: 0.4878
Epoch 4/5
74/74 [==============================] - 53s 713ms/step - loss: 0.7306 - accuracy: 0.4989 - val_loss: 0.7175 - val_accuracy: 0.5087
Epoch 5/5
74/74 [==============================] - 53s 715ms/step - loss: 0.7135 - accuracy: 0.5133 - val_loss: 0.7064 - val_accuracy: 0.5104



Create the ResNet50 Model:
Defines a ResNet50-based model by initializing it with pre-trained weights and excluding the top layers. The last 30 layers of the base model are unfrozen for fine-tuning.

Adding Custom Layers:
Adds a global average pooling layer, a dense layer with 256 units and ReLU activation, and a dropout layer to prevent overfitting. The final layer is a dense layer with softmax activation for classification.

Compile the Model:
Compiles the model using the Adam optimizer with a learning rate of 0.001 and categorical cross-entropy loss, setting up the model for training with accuracy as the evaluation metric.

Model Instantiation:
Creates an instance of the ResNet50 model with the defined architecture and compilation settings.

In [15]:

# Create the ResNet50 model
def create_resnet50_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    #Unfreeze the top layers of the base model
    for layer in base_model.layers[-30:]:  # Unfreeze the last 30 layers
        layer.trainable = True
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model
    opt = Adam(learning_rate=0.001)
    l2_reg=0.001
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    
    return model

model = create_resnet50_model()




ReduceLROnPlateau Callback:
Sets up a learning rate reduction strategy to lower the learning rate by a factor of 0.2 if the validation loss does not improve for 3 epochs, with a minimum learning rate of \(1 \times 10^{-6}\).

EarlyStopping Callback:
Implements early stopping to halt training if the validation loss does not improve for 5 consecutive epochs, restoring the best weights observed during training.

ModelCheckpoint Callback:
Configures a checkpointing strategy to save the model's weights to a file named `best_model.h5` whenever the validation loss achieves a new minimum, ensuring the best model is preserved.

Train the Model:
Trains the ResNet50 model using the training data generator for 5 epochs, validating against the test data generator, and applying the specified callbacks to optimize and save the best model during training.

In [16]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

# Train the model
history_resnet50 = model.fit(
    train_generator,
    epochs=5,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping, checkpoint])
    

Epoch 1/5
75/75 [==============================] - ETA: 0s - loss: 0.5944 - accuracy: 0.7785

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


75/75 [==============================] - 96s 799ms/step - loss: 0.5944 - accuracy: 0.7785 - val_loss: 1.4272 - val_accuracy: 0.5142 - lr: 0.0010
Epoch 2/5
75/75 [==============================] - 53s 709ms/step - loss: 0.2569 - accuracy: 0.8959 - val_loss: 2.3894 - val_accuracy: 0.4858 - lr: 0.0010
Epoch 3/5
75/75 [==============================] - 63s 838ms/step - loss: 0.2104 - accuracy: 0.9219 - val_loss: 0.8714 - val_accuracy: 0.4858 - lr: 0.0010
Epoch 4/5
75/75 [==============================] - 53s 697ms/step - loss: 0.2103 - accuracy: 0.9185 - val_loss: 1.4248 - val_accuracy: 0.4858 - lr: 0.0010
Epoch 5/5
75/75 [==============================] - 57s 763ms/step - loss: 0.1806 - accuracy: 0.9344 - val_loss: 1.8515 - val_accuracy: 0.4858 - lr: 0.0010




Create the EfficientNet Model:
Defines an EfficientNetB0-based model by initializing it with pre-trained weights and excluding the top layers. The last 30 layers of the base model are unfrozen for fine-tuning.

Adding Custom Layers:
Adds a global average pooling layer, batch normalization, a dense layer with 256 units and ReLU activation, and a dropout layer to prevent overfitting. The final layer is a dense layer with softmax activation for classification.

Compile the Model:
Compiles the model using the RMSprop optimizer with a learning rate of 0.0001 and categorical cross-entropy loss, setting up the model for training with accuracy as the evaluation metric.

Model Instantiation:
Creates an instance of the EfficientNet model with the defined architecture and compilation settings.

In [17]:


# Create the EfficientNet model
def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Unfreeze the top layers of the base model
    for layer in base_model.layers[-30:]:  # Unfreeze the last 30 layers
        layer.trainable = True
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model with RMSprop optimizer
    opt = RMSprop(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

efficientnet_model = create_efficientnet_model()







ReduceLROnPlateau Callback:
Configures a learning rate reduction strategy to lower the learning rate by a factor of 0.2 if the validation loss does not improve for 3 epochs, with a minimum learning rate of \(1 \times 10^{-5}\) and verbose output.

EarlyStopping Callback:
Sets up early stopping to halt training if the validation loss does not improve for 5 consecutive epochs, restoring the best weights observed during training, with verbose output for progress tracking.

ModelCheckpoint Callback:
Implements a checkpointing strategy to save the model's weights to a file named `best_efficientnet_model.h5` whenever the validation loss achieves a new minimum, ensuring the best model is preserved, with verbose output.

Train the Model:
Trains the EfficientNet model using the training data generator for 5 epochs, validating against the validation data generator, and applying the specified callbacks to optimize and save the best model during training.

In [18]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('best_efficientnet_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Train the model
history_efficientnet = efficientnet_model.fit(
    train_generator,
    epochs=5,  # Increase the number of epochs
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)


Epoch 1/5


2024-06-22 01:50:33.843661: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_4/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


75/75 [==============================] - ETA: 0s - loss: 1.1131 - accuracy: 0.5537
Epoch 1: val_loss improved from inf to 0.90728, saving model to best_efficientnet_model.h5
75/75 [==============================] - 78s 738ms/step - loss: 1.1131 - accuracy: 0.5537 - val_loss: 0.9073 - val_accuracy: 0.5142 - lr: 1.0000e-04
Epoch 2/5
75/75 [==============================] - ETA: 0s - loss: 0.5283 - accuracy: 0.8003
Epoch 2: val_loss improved from 0.90728 to 0.86565, saving model to best_efficientnet_model.h5
75/75 [==============================] - 54s 719ms/step - loss: 0.5283 - accuracy: 0.8003 - val_loss: 0.8656 - val_accuracy: 0.5142 - lr: 1.0000e-04
Epoch 3/5
75/75 [==============================] - ETA: 0s - loss: 0.2825 - accuracy: 0.8922
Epoch 3: val_loss did not improve from 0.86565
75/75 [==============================] - 53s 700ms/step - loss: 0.2825 - accuracy: 0.8922 - val_loss: 0.9490 - val_accuracy: 0.5142 - lr: 1.0000e-04
Epoch 4/5
75/75 [==============================] - 


Function Definition:
Defines the function `ensemble_predict` to make predictions using an ensemble of models and a meta-model for final classification.

Concatenate Predictions:
Concatenates the predictions from each model in the ensemble for the test data along the second axis, forming a single feature set for training the meta-model.

Train Meta-Model:
Trains a logistic regression model (meta-model) using the concatenated predictions as input features and the true labels from the test generator.

Predict with Meta-Model:
Concatenates the predictions from each model again and uses the trained logistic regression meta-model to predict the final labels for the test data.

Return Predictions:
Returns the final predicted labels from the logistic regression meta-model for the test data.

In [19]:
def ensemble_predict(models, test_generator):
    X = np.concatenate([model.predict(test_generator) for model in models], axis=1)
    y = test_generator.classes
    
    meta_model = LogisticRegression(max_iter=1000)
    meta_model.fit(X, y)
    
    X_test = np.concatenate([model.predict(test_generator) for model in models], axis=1)
    y_pred = meta_model.predict(X_test)
    
    return y_pred

Function Definition:
Defines the function `get_predictions` to generate binary predictions from a given model and data generator.

Model Predictions:
Uses the model to predict probabilities for the input data provided by the data generator.

Binary Conversion:
Converts the predicted probabilities to binary labels (0 or 1) by applying a threshold of 0.5, and returns these binary predictions as integers.

In [20]:
def get_predictions(model, data_generator):
    predictions = model.predict(data_generator)
    return (predictions > 0.5).astype(int)



Baseline Model Predictions:
Generates binary predictions from the `baseline_model` using the `get_predictions` function and the `test_generator`.

ResNet50 Model Predictions:
Generates binary predictions from the `resnet50_model` using the `get_predictions` function and the `test_generator`.

EfficientNet Model Predictions:
Generates binary predictions from the `efficientnet_model` using the `get_predictions` function and the `test_generator`.

In [21]:
baseline_preds = get_predictions(baseline_model, test_generator)
resnet50_preds = get_predictions(resnet50_model, test_generator)
efficientnet_preds = get_predictions(efficientnet_model, test_generator)

19/19 [==============================] - 8s 345ms/step


Averaging Predictions:
Calculates the average of the binary predictions from the baseline, ResNet50, and EfficientNet models by summing their predictions and dividing by three.

Rounding to Binary:
Rounds the averaged predictions to the nearest integer (0 or 1) to obtain the final ensemble binary predictions, combining the strengths of all three models.


In [22]:
ensemble_preds = np.round((baseline_preds + resnet50_preds + efficientnet_preds) / 3)


Model List:
Creates a list of models (`baseline_model`, `resnet50_model`, and `efficientnet_model`) to be used in the ensemble.

Ensemble Predictions:
Calls the `ensemble_predict` function with the list of models and the `test_generator`, generating the final ensemble predictions using the meta-model approach.

In [23]:
models = [baseline_model, resnet50_model, efficientnet_model]
ensemble_predictions = ensemble_predict(models, test_generator)

19/19 [==============================] - 7s 349ms/step


Correct Labels:
Retrieves the true labels for the test dataset from the `test_generator`.

Calculate Ensemble Accuracy:
Uses the `accuracy_score` function from Scikit-learn to compute the accuracy of the ensemble predictions by comparing them to the correct labels.

Print Accuracy:
Prints the ensemble accuracy, formatted to two decimal places, to provide a performance measure of the ensemble model.

In [25]:
correct_labels = test_generator.classes
ensemble_accuracy = accuracy_score(correct_labels, ensemble_predictions)
print(f'Ensemble Accuracy: {ensemble_accuracy:.2f}')

Ensemble Accuracy: 0.51



Model Training Loop:
Sets up a loop to train three instances of the baseline model, each with different initialization but the same architecture.

Create Baseline Model:
Creates an instance of the baseline model with specified input shape and number of classes.

Early Stopping Callback:
Sets up early stopping to monitor validation loss, halting training if it doesn't improve for 10 epochs, and restores the best model weights.

Model Training:
Trains each model using the training data generator, validating with the validation data generator, and applying the early stopping callback over a specified number of epochs and steps per epoch.

Save Model Weights:
Saves the weights of each trained model to a file, naming them uniquely based on the loop index to distinguish between different model instances.

In [28]:
num_models = 3
for i in range(num_models):
    model = create_baseline_model(input_shape=input_shape, num_classes=num_classes)
    baseline_preds_{i} = get_predictions(model, test_generator)
    
    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.n // batch_size,
        callbacks=[early_stopping]
    )
    
    # Save model weights
    model.save_weights(f'baseline_model_weights_{i}.h5')


Epoch 1/5
74/74 [==============================] - 51s 654ms/step - loss: 3.3764 - accuracy: 0.6535 - val_loss: 4.0051 - val_accuracy: 0.5191
Epoch 2/5
74/74 [==============================] - 49s 663ms/step - loss: 2.5584 - accuracy: 0.7747 - val_loss: 2.3609 - val_accuracy: 0.7604
Epoch 3/5
74/74 [==============================] - 50s 679ms/step - loss: 2.1457 - accuracy: 0.7692 - val_loss: 1.9953 - val_accuracy: 0.7309
Epoch 4/5
74/74 [==============================] - 50s 676ms/step - loss: 1.8074 - accuracy: 0.8107 - val_loss: 1.6514 - val_accuracy: 0.7882
Epoch 5/5
74/74 [==============================] - 50s 678ms/step - loss: 1.5641 - accuracy: 0.8251 - val_loss: 1.4737 - val_accuracy: 0.8194
Epoch 1/5
74/74 [==============================] - 53s 681ms/step - loss: 3.2040 - accuracy: 0.6739 - val_loss: 11.4639 - val_accuracy: 0.5174
Epoch 2/5
74/74 [==============================] - 49s 664ms/step - loss: 2.5483 - accuracy: 0.7632 - val_loss: 3.5796 - val_accuracy: 0.5208
Epoch



Initialize Model List:
Creates a list to store multiple instances of the baseline model.

Load Model Weights:
Loops through to create instances of the baseline model and loads pre-trained weights from saved files for each instance, ensuring multiple versions of the model are ready for ensemble prediction.

Model Predictions:
Generates predictions from all model instances on the test data, storing the results in a list.

Average Predictions:
Calculates the ensemble predictions by averaging the probabilities from all model predictions.

Determine Final Labels:
Determines the final predicted labels by selecting the class with the highest average probability for each test sample.

Calculate Ensemble Accuracy:
Computes the accuracy of the ensemble model by comparing the final predicted labels to the true labels from the test data.

Print Ensemble Accuracy:
Outputs the calculated accuracy of the ensemble model, formatted to two decimal places.

In [31]:
# Train instances of multiple models 
num_models = 3
models = []
for i in range(num_models):
    model = create_baseline_model(input_shape=input_shape, num_classes=num_classes)
    # Load pre-trained weights if available, or train here
    model.load_weights(f'baseline_model_weights_{i}.h5')  # Ensure you have trained and saved these models
    models.append(model)

# Predict with all models
predictions = [model.predict(test_generator, steps=np.ceil(test_generator.n / test_generator.batch_size), verbose=1) for model in models]

# Ensemble predictions (average the probabilities)
ensemble_preds = np.mean(predictions, axis=0)
ensemble_preds_labels = np.argmax(ensemble_preds, axis=1)

# Calculate accuracy
correct_labels = test_generator.classes
ensemble_accuracy = accuracy_score(correct_labels[:len(ensemble_preds_labels)], ensemble_preds_labels)
print(f'Ensemble Accuracy: {ensemble_accuracy:.2f}')


19/19 [==============================] - 6s 305ms/step
Ensemble Accuracy: 0.47


In [33]:

ensemble_preds = np.round((baseline_preds_1 + baseline_preds_2 + baseline_preds_3) / 3)


NameError: name 'baseline_preds_1' is not defined

In [ ]:
correct_labels = test_generator.classes
ensemble_accuracy = accuracy_score(correct_labels, ensemble_predictions)
print(f'Ensemble Accuracy: {ensemble_accuracy:.2f}')